In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [98]:
class Backtester():
    def __init__(self, symbol, balance, comission, data):
        self.symbol = symbol
        self.initial_balance = balance
        self.current_balance = balance
        self.comission = comission / 100
        self.data = data
        self.units = 0.0
        self.trades = 0

    def get_bar(self, bar):
        date = str(self.data.index[bar].date())
        price = self.data["Close"].iloc[bar]
        return date, price

    def is_valid_amount(self, bar, units):
        date, price = self.get_bar(bar)
        amount = price * units * (1 + self.comission)
        if amount > self.current_balance:
            print("No tienes suficiente dinero para comprar {} unidades de {} en la fecha {}".format(units, self.symbol, date))
            return False
        return True

    def buy_instrument(self, bar, units):
        date, price = self.get_bar(bar)
        if not self.is_valid_amount(bar, units):
            return
        self.units += units
        cost = units * price
        comission_cost = cost * self.comission
        self.current_balance -= cost * (1 + self.comission)
        self.trades += 1
        print("Compraste {} unidades de {} en la fecha {} a un precio de $ {} USD por un total de $ {} USD con una comisión de $ {} USD; Total: {}".format(units, self.symbol, date, price, cost, comission_cost, cost + comission_cost))
    
    def sell_units(self, bar, units = None):
        date, price = self.get_bar(bar)
        if units > self.units:
            print("No tienes suficientes unidades para vender {} unidades de {} en la fecha {}".format(units, self.symbol, date))
            return
        self.units -= units
        self.current_balance += units * price * (1 - self.comission)
        self.trades += 1
        print("Vendiste {} unidades de {} en la fecha {} a un precio de $ {} USD".format(units, self.symbol, date, price))

    def show_current_balance(self, bar):
        date, price = self.get_bar(bar)
        print("Tú balance actual es de $ {} USD en la fecha {}".format(self.current_balance, date))

    def show_current_position_value(self, bar):
        date, price = self.get_bar(bar)
        print("Tú posición actual es de $ {} USD en la fecha {}".format(self.units * price, date))

    def show_net_asset_value(self, bar):
        date, price = self.get_bar(bar)
        print("Tú valor neto actual es de $ {} USD en la fecha {}".format(self.current_balance + self.units * price, date))

    def show_stats(self, bar):
        date, price = self.get_bar(bar)
        performance = (self.current_balance - self.initial_balance) / self.initial_balance * 100
        print("Tú balance inicial fue de $ {} en la fecha {} y tu balance actual es de $ {} USD en la fecha {}".format(self.initial_balance, self.data.index[0].date(), self.current_balance, date))
        print("Tú rendimiento es de {} %".format(performance))
        print("Tú número de operaciones es de {}".format(self.trades))
        self.show_current_position_value(bar)
        self.show_net_asset_value(bar)

In [99]:
data = pd.read_csv("resources/BTCUSD_1D.csv", index_col="Date", parse_dates=True)

In [100]:
bc = Backtester("BTCUSD", 10000, 0.1, data)

In [101]:
bc.data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-09-20,3916.360107,4031.389893,3857.729980,3905.949951,3905.949951,1213830016
2017-09-21,3901.469971,3916.419922,3613.629883,3631.040039,3631.040039,1411480064
2017-09-22,3628.020020,3758.270020,3553.530029,3630.699951,3630.699951,1194829952
2017-09-23,3629.919922,3819.209961,3594.580078,3792.399902,3792.399902,928113984
2017-09-24,3796.149902,3796.149902,3666.899902,3682.840088,3682.840088,768014976
...,...,...,...,...,...,...
2022-09-17,19777.033203,20162.531250,19777.033203,20127.576172,20127.576172,24957448100
2022-09-18,20127.234375,20127.234375,19387.492188,19419.505859,19419.505859,31254779144
2022-09-19,19418.572266,19639.480469,18390.318359,19544.128906,19544.128906,40177002624


In [102]:
bc.comission

0.001

In [103]:
bc.current_balance, bc.initial_balance

(10000, 10000)

In [104]:
bc.buy_instrument(0, 1)

Compraste 1 unidades de BTCUSD en la fecha 2017-09-20 a un precio de $ 3905.949951 USD por un total de $ 3905.949951 USD con una comisión de $ 3.905949951 USD; Total: 3909.855900951


In [105]:
# bc.sell_units(3, 1)

In [106]:
bc.current_balance, bc.initial_balance

(6090.144099049001, 10000)

In [107]:
bc.show_current_position_value(3)

Tú posición actual es de $ 3792.399902 USD en la fecha 2017-09-23


In [108]:
bc.show_current_balance(3)

Tú balance actual es de $ 6090.144099049001 USD en la fecha 2017-09-23


In [109]:
bc.show_net_asset_value(3)

Tú valor neto actual es de $ 9882.544001049002 USD en la fecha 2017-09-23


In [110]:
bc.sell_units(3, 1)

Vendiste 1 unidades de BTCUSD en la fecha 2017-09-23 a un precio de $ 3792.399902 USD


In [111]:
bc.show_current_balance(3)

Tú balance actual es de $ 9878.751601147002 USD en la fecha 2017-09-23


In [112]:
bc.show_stats(3)

Tú balance inicial fue de $ 10000 en la fecha 2017-09-20 y tu balance actual es de $ 9878.751601147002 USD en la fecha 2017-09-23
Tú rendimiento es de -1.2124839885299843 %
Tú número de operaciones es de 2
Tú posición actual es de $ 0.0 USD en la fecha 2017-09-23
Tú valor neto actual es de $ 9878.751601147002 USD en la fecha 2017-09-23


In [113]:
df_strategy = pd.read_csv("resources/BTCUSD_1D.csv", index_col="Date", parse_dates=True)

In [116]:
df_strategy["Returns"] = np.log(df_strategy["Close"].pct_change() + 1)

In [117]:
df_strategy.head()

,Open,High,Low,Close,Adj Close,Volume,Returns
Date,,,,,,,
2017-09-20,3916.360107,4031.389893,3857.729980,3905.949951,3905.949951,1213830016,NaN
2017-09-21,3901.469971,3916.419922,3613.629883,3631.040039,3631.040039,1411480064,-0.072982
2017-09-22,3628.020020,3758.270020,3553.530029,3630.699951,3630.699951,1194829952,-0.000094
2017-09-23,3629.919922,3819.209961,3594.580078,3792.399902,3792.399902,928113984,0.043574
2017-09-24,3796.149902,3796.149902,3666.899902,3682.840088,3682.840088,768014976,-0.029315


In [118]:
df_strategy["SMA 50"] = df_strategy["Close"].rolling(50).mean()
df_strategy["SMA 200"] = df_strategy["Close"].rolling(200).mean()

In [119]:
df_strategy.head()

,Open,High,Low,Close,Adj Close,Volume,Returns,SMA 50,SMA 200
Date,,,,,,,,,
2017-09-20,3916.360107,4031.389893,3857.729980,3905.949951,3905.949951,1213830016,NaN,NaN,NaN
2017-09-21,3901.469971,3916.419922,3613.629883,3631.040039,3631.040039,1411480064,-0.072982,NaN,NaN
2017-09-22,3628.020020,3758.270020,3553.530029,3630.699951,3630.699951,1194829952,-0.000094,NaN,NaN
2017-09-23,3629.919922,3819.209961,3594.580078,3792.399902,3792.399902,928113984,0.043574,NaN,NaN
2017-09-24,3796.149902,3796.149902,3666.899902,3682.840088,3682.840088,768014976,-0.029315,NaN,NaN


In [121]:
df_strategy.dropna(inplace=True)

In [122]:
df_strategy.head()

,Open,High,Low,Close,Adj Close,Volume,Returns,SMA 50,SMA 200
Date,,,,,,,,,
2018-04-07,6630.509766,7050.540039,6630.509766,6911.089844,6911.089844,3976610048,0.040570,9102.242607,9588.698008
2018-04-08,6919.979980,7111.560059,6919.979980,7023.520020,7023.520020,3652499968,0.016137,9020.459004,9604.285858
2018-04-09,7044.319824,7178.109863,6661.990234,6770.729980,6770.729980,4894060032,-0.036656,8944.837607,9619.984308
2018-04-10,6795.439941,6872.410156,6704.149902,6834.759766,6834.759766,4272750080,0.009412,8857.026807,9636.004607
2018-04-11,6843.470215,6968.319824,6817.589844,6968.319824,6968.319824,4641889792,0.019353,8768.319199,9651.884207


In [123]:
bc2 = Backtester("BTCUSD", 10000, 0.1, df_strategy)

In [124]:
for bar in range(len(bc2.data)):
    if bc2.data["SMA 50"].iloc[bar] > bc2.data["SMA 200"].iloc[bar]:
        bc2.buy_instrument(bar, 0.1)
    else:
        if bc2.units >= 0.1:
            bc2.sell_units(bar, bc2.units)

Compraste 0.1 unidades de BTCUSD en la fecha 2019-04-24 a un precio de $ 5464.866699 USD por un total de $ 546.4866699 USD con una comisión de $ 0.5464866699 USD; Total: 547.0331565699
Compraste 0.1 unidades de BTCUSD en la fecha 2019-04-25 a un precio de $ 5210.515625 USD por un total de $ 521.0515625 USD con una comisión de $ 0.5210515625000001 USD; Total: 521.5726140625001
Compraste 0.1 unidades de BTCUSD en la fecha 2019-04-26 a un precio de $ 5279.348145 USD por un total de $ 527.9348145 USD con una comisión de $ 0.5279348145 USD; Total: 528.4627493145
Compraste 0.1 unidades de BTCUSD en la fecha 2019-04-27 a un precio de $ 5268.291016 USD por un total de $ 526.8291016000001 USD con una comisión de $ 0.5268291016000001 USD; Total: 527.3559307016001
Compraste 0.1 unidades de BTCUSD en la fecha 2019-04-28 a un precio de $ 5285.13916 USD por un total de $ 528.513916 USD con una comisión de $ 0.528513916 USD; Total: 529.042429916
Compraste 0.1 unidades de BTCUSD en la fecha 2019-04-29

In [125]:
bc2.show_stats(len(bc2.data) - 1)

Tú balance inicial fue de $ 10000 en la fecha 2018-04-07 y tu balance actual es de $ 39751.201128752604 USD en la fecha 2022-09-21
Tú rendimiento es de 297.51201128752604 %
Tú número de operaciones es de 114
Tú posición actual es de $ 5.275595498788199e-13 USD en la fecha 2022-09-21
Tú valor neto actual es de $ 39751.201128752604 USD en la fecha 2022-09-21
